## Sacar información de la película

In [1]:
import urllib.request
import re

class_datos = 'ipc-metadata-list-item__content-container' # Dentro de un div
class_rating = 'sc-d541859f-1 imUuxf' # Dentro de un span
# Coger los 3 primeros (direccion, guionistas, elenco)
class_sinopsis = 'sc-42125d72-1 igbBrx'
class_numeroVotos = 'sc-d541859f-3 dwhNqC'
class_duracion = 'ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt' # Tiene dentro 3 li con año, clasificación y duración

In [2]:
CLASS_DURACION = 'ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt'  # Tiene dentro 3 li con año, clasificación y duración
CLASS_RATING = 'sc-d541859f-1 imUuxf'  # Dentro de un span
CLASS_NUMEROVOTOS = 'sc-d541859f-3 dwhNqC' # Dentro de un div
CLASS_SINOPSIS = 'sc-42125d72-1 igbBrx'
CLASS_DATOS = 'ipc-metadata-list-item__content-container'  # Dentro de un div
# Coger los 3 primeros (direccion, guionistas, elenco)


# Definir la URL
url = 'https://www.imdb.com/es/title/tt27722575/?ref_=nv_sr_srsg_0_tt_8_nm_0_in_0_q_200'

# Definir los headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Crear una petición con los headers
req = urllib.request.Request(url, headers=headers)

# Descargar el HTML
with urllib.request.urlopen(req) as response:
    html = response.read().decode('utf-8')

# Imprimir el HTML descargado
print(response.status)




200


In [ ]:
# Sacamos (Año), (Calificación), (Duración)
extractor = rf'.*?\<ul class="{CLASS_DURACION}".+?\>\<li.+?\>|<a.+?\>([\d]+?)\<.+?\<li.+?\>|<a.+?\>([^<]+?)\<.+?\<li.+?\>|<a.+?\>([^<]+?)\<.+?' 
# Sacamos (puntuación)
extractor += rf'\<span class="{CLASS_RATING}"\>([\d]+?)\<.+?'
# Sacamos (nº de votos)
extractor += rf'\<div class="{CLASS_NUMEROVOTOS}"\>([\d]+?)\<.+?'
# Sacamos (sinopsis)
extractor += rf'\<span .+?class="{CLASS_SINOPSIS}"\>([^<]+?)\<.+?'

anio, calificacion, duracion, puntuacion, num_votos, sinopsis = re.findall(extractor, html)
print(anio, calificacion, duracion, puntuacion, num_votos, sinopsis)

In [ ]:
# Sacamos (direccion), (guionistas), (elenco) con un findall
informacion = rf'\<div class="{CLASS_DATOS}"\>\<ul.+?\>\<li.+?\>\<a.+?\>"([^<]+?)"\<.+?'

datos = re.findall(informacion, html)